In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore', module="h5py")

In [ ]:
import pickle
import numpy as np
import itertools
from types import SimpleNamespace
import copy
from mpnetcdf4.utils import Timer
from mpnetcdf4.benchmarking import with_stats, plot_benchmark_results
from mpnetcdf4 import ReaderFactory
from tests import TEST_HDF_FILE, TEST_HDF_STACKED_FILE

display(TEST_HDF_STACKED_FILE)
!hostname && free -m

In [ ]:
def run_bencmark(pp):
    print(pp)
    
    with Timer() as t_total:
        with Timer(message='Prepare') as t:
            mp_factory = ReaderFactory(pp.nprocs, mb=pp.mb)
        
        t_prepare = t.elapsed
    
        with Timer(message='Open') as t:
            f = mp_factory.open(pp.fname)
    
        t_open = t.elapsed
        
        read = with_stats(f.read, message='Read x%d'%pp.nprocs)
    
        out, stats = read(measurements=pp.measurements,
                          src_roi=pp.src_roi,
                          chunk_scale=pp.chunk_scale)
    
    stats.t_total = t_total.elapsed
    stats.t_prepare = t_prepare
    stats.t_open = t_open
    stats.params = copy.copy(pp)
    
    return out, stats

In [ ]:
params = SimpleNamespace(
    fname=TEST_HDF_STACKED_FILE,
    mb=100,
    nprocs=1,
    src_roi=None, #np.s_[:, 800:1000, 200:400],
    chunk_scale=(1,1,8),
    measurements=['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'coastal_aerosol'])


In [ ]:
all_stats = []
for num_procs in range(1,16):
    pp = copy.copy(params)
    pp.nprocs = num_procs
    out, stats = run_bencmark(pp)
    del out
    all_stats.append(stats)

In [ ]:
plot_benchmark_results(all_stats, fig=plt.figure(figsize=(16,4)))

In [ ]:
[ (s.params.nprocs, s.t_total, 100*(s.t_total - s.elapsed)/s.t_total, s.throughput_mb) for s in all_stats]

In [ ]:
pickle.dump(all_stats, open('all_stats-6.pickle', 'wb'))

In [ ]:
!ls -la *pickle